---

<!-- <a href="https://github.com/rraadd88/roux/blob/master/examples/roux_workflow_io.ipynb"><img align="right" style="float:right;" src="https://img.shields.io/badge/-source-cccccc?style=flat-square"></a>
 -->
 
## ⚙️🗺️ Reading multiple configs.

In [ ]:
# install extra requirements
# !pip install roux[workflow]

### Demo inputs

In [1]:
input_dir_path='tests/input/roux_workflow_io/'

In [2]:
## creating the demo configs
configs=dict(
    ## meta-config that contains other configs
    meta_config="""
    key1: value1
    key2: value2
    key3: interpolated from metaconfig = ${key2}
    
    ## append other configs
    sub_config1:
        config_path: '"""+input_dir_path+"""sub_config1.yaml' # should be one key
        path: path/to/file1
        description: sample1. 
    sub_config2:
        config_path: '"""+input_dir_path+"""sub_config2.yaml' # path should exist
        path: path/to/file2
        description: sample2. 
    """,
    ## individual configs to be appended
    sub_config1="""
    c1key1: value from metaconfig = ${key1} # interpolated from c0
    c1key2: value2
    c1key3: value3
    """,

    sub_config2="""
    c2key1: value from metaconfig = ${key1} # interpolated from c0
    c2key2: value interpolated in config1 = ${sub_config1.c1key1} # interpolated from c1
    c2key3: value3
    """,
    )
import yaml
configs={k:yaml.safe_load(configs[k]) for k in configs}
from pprint import pprint
pprint({k:d for k,d in configs.items()})

{'meta_config': {'key1': 'value1',
                 'key2': 'value2',
                 'key3': 'interpolated from metaconfig = ${key2}',
                 'sub_config1': {'config_path': 'tests/input/roux_workflow_io/sub_config1.yaml',
                                 'description': 'sample1.',
                                 'path': 'path/to/file1'},
                 'sub_config2': {'config_path': 'tests/input/roux_workflow_io/sub_config2.yaml',
                                 'description': 'sample2.',
                                 'path': 'path/to/file2'}},
 'sub_config1': {'c1key1': 'value from metaconfig = ${key1}',
                 'c1key2': 'value2',
                 'c1key3': 'value3'},
 'sub_config2': {'c2key1': 'value from metaconfig = ${key1}',
                 'c2key2': 'value interpolated in config1 = '
                           '${sub_config1.c1key1}',
                 'c2key3': 'value3'}}


In [3]:
## save the demo configs
from roux.lib.io import to_dict
[to_dict(configs[k],f'tests/input/roux_workflow_io/{k}.yaml') for k in configs]

['tests/input/roux_workflow_io/meta_config.yaml',
 'tests/input/roux_workflow_io/sub_config1.yaml',
 'tests/input/roux_workflow_io/sub_config2.yaml']

### Reading the meta-config

In [4]:
from roux.workflow.io import read_metadata 
config_base=read_metadata(
    f'{input_dir_path}/meta_config.yaml', # path to the meta-config
    verbose=True, # verbose
    )

### Output

In [5]:
config_base

{'key1': 'value1',
 'key2': 'value2',
 'key3': 'interpolated from metaconfig = value2',
 'sub_config1': {'config_path': 'tests/input/roux_workflow_io/sub_config1.yaml',
  'description': 'sample1.',
  'path': 'path/to/file1',
  'c1key1': 'value from metaconfig = value1',
  'c1key2': 'value2',
  'c1key3': 'value3'},
 'sub_config2': {'config_path': 'tests/input/roux_workflow_io/sub_config2.yaml',
  'description': 'sample2.',
  'path': 'path/to/file2',
  'c2key1': 'value from metaconfig = value1',
  'c2key2': 'value interpolated in config1 = value from metaconfig = value1',
  'c2key3': 'value3'}}

In [6]:
## test for testbook
assert config_base['sub_config2']['c2key2']=='value interpolated in config1 = value from metaconfig = value1', config_base['sub_config2']['c2key2']
assert len(config_base)==5, len(config_base)
assert len(config_base['sub_config2'])==6, config_base['sub_config2']
print(config_base['sub_config2']['c2key2'])

value interpolated in config1 = value from metaconfig = value1


### Modifying config using `inputs` parameter

In [7]:
from roux.workflow.io import read_metadata 
config_base=read_metadata(
    f'{input_dir_path}/meta_config.yaml', # path to the meta-config
    inputs=dict(key1='modified'),
    verbose=True, # verbose
    )

In [8]:
## test for testbook
assert config_base['sub_config2']['c2key2']=='value interpolated in config1 = value from metaconfig = modified', config_base['sub_config2']['c2key2']
assert len(config_base)==5, len(config_base)
assert len(config_base['sub_config2'])==6, config_base['sub_config2']
print(config_base['sub_config2']['c2key2'])

value interpolated in config1 = value from metaconfig = modified
